
<h1>Simulation Simple Iteration "Cimulation" Israel Bat Yam Version March 6</h1>
this is a proof of concept that the simulation works and that it's solid.
this simultation doesn't hold all the correct data, as the data is fautlty and rework is needed to it
what this simulation has is 5 iterations
the first 3 iterations are Tama 38/1
the 4th iteration is Tama 38/2
and the 5th iteration is an example of a Taba Planning

<h1>step 1: Loading the files</h1>

In [1]:
import pandas as pd
import numpy as np
import json
import os
#install geopandas library for geogrpahic operation
import geopandas as gpd
import ipywidgets as widgets
import numpy as np
from datetime import date
import folium
from datetime import datetime
from IPython.display import HTML, display

## Reference to the dictionary File
### Dictionary File is important to keep a reference of the main files for the simulation

In [3]:
#reference dictionary file
path_1 = 'library.json'
with open(path_1) as json_file:
    data = json.load(json_file)

json_formatted_str = json.dumps(data, indent=2)
print(json_formatted_str)

#rules dictionary file file
path_2 = 'rules.json'
with open(path_2) as json_file:
    rules_1 = json.load(json_file)


json_formatted_str = json.dumps(rules_1, indent=2)
print('-----Rules File-------')
print(json_formatted_str)

{
  "GIS_package": {
    "old_bldgs": {
      "package": "Source/data/GIS/BaseLayers.gpkg",
      "layer": "bldgs_renewal_2020"
    },
    "old_bldgs_background": {
      "package": "Source/data/GIS/BaseLayers.gpkg",
      "layer": "old_bldgs_all_israel_tm"
    }
  },
  "script_file": "Source/data/script/script_ssi_march_9_1035pm.xlsx",
  "script_file_old": "Source/data/script/script_ssi_march_8_535pm.xlsx",
  "future_script": "Source/data/FutureBldgs_Tama_38.xlsx",
  "Buildings_ref": "Source/data/Buildings_flat_march_10",
  "bldg_path": "Source/data/bldg/",
  "Agents": "Source/Agents_8pm.xlsx",
  "Google Drive Prepath": "content/drive/My Drive/BatYam NY DRIVE/Simulations/March 4 2020 After NYC/"
}
-----Rules File-------
{
  "Maintenance": {
    "old_bldgs": {
      "4": "100",
      "8": "180",
      "12": "200"
    },
    "new_bldgs": {
      "4": "250",
      "8": "320",
      "12": "400",
      "15": "450"
    }
  },
  "Income": {
    "Low": {
      "0": "10000"
    },
    "Medium"

In [4]:
pack_old_bldg = data["GIS_package"]["old_bldgs"]["package"]
layer_old_bldg = data["GIS_package"]["old_bldgs"]["layer"]

pack_all_old_bldg = data["GIS_package"]["old_bldgs_background"]["package"]
layer_all_old_bldg = data["GIS_package"]["old_bldgs_background"]["layer"]

In [5]:
old_bldgs = gpd.read_file(pack_old_bldg,layer = layer_old_bldg,driver='GPKG')
old_all_bldgs = gpd.read_file(pack_all_old_bldg,layer = layer_all_old_bldg,driver='GPKG')

In [6]:
old_bldgs.head(2)

,Area,floors,height,bld_num,newUnits,polyID,bld_row_num,rowNum,bld_id,OriginalUnits,AdditionalUnits,TotalUnits,geometry
0,380.365771,2,6.0,None,8.0,3,53,53,212_59,NaN,NaN,NaN,"POLYGON ((175626.182 659083.328, 175633.042 65..."
1,435.489811,4,12.0,None,16.0,6,50,50,212_61,NaN,NaN,NaN,"POLYGON ((175615.252 659026.518, 175616.122 65..."


In [7]:
#master script
script_file = pd.read_excel(data["script_file"])

In [8]:
script_file

,index,bld_address,OriginalUnits,OriginalFloors,OriginalHouseSize,purchase_p,rent_price,Maintenace,ArnonaPerMeter,Arnona,...,bld_operation,pointPolygon,ParcelChange,FuturePlanID,ExcelBefore,ExcelChange,ExcelPolygon,GIS_files,Address Title,TypeTitle
0,0,215_22,18,3.0,88,22930,62,100,5.4358,478,...,1,Point,No,20140882,before_215_22.xlsx,after_215_22.xlsx,NaN,bldgs_1a,"סוקולוב 22, בת ים",תמ״א 38/1
1,1,216_10,12,3.0,123,24376,68,100,5.4358,668,...,1,Point,No,20120398,before_216_10.xlsx,after_216_10.xlsx,NaN,bldgs_2a,"מסריק 10, בת ים",תמ״א 38/1
2,2,216_4,12,3.0,94,23351,67,100,5.4358,510,...,1,Point,No,20141086,before_216_4.xlsx,after_216_4.xlsx,NaN,bldgs_3a,"מסריק 4, בת ים",תמ״א 38/1
3,3,209_6,6,3.0,94,22922,50,100,5.4358,510,...,2,Point,No,20140231,before_209_6.xlsx,after_209_6.xlsx,NaN,bldgs_6a,"ארלוזורוב 9, בת ים",תמ״א 38/2
4,4,complex_4,46,2.5,72,21336,62,100,5.4358,391,...,3,Polygon,Yes,502-0317495,before_complex_4.xlsx,after_complex_4.xlsx,polygon_502_0317495.xlsx,bldgs_45a,"הרצל 16,18 וחנה סנש 11, 9, 7, בת ים",תב״ע פינוי בינוי


In [10]:
agents_name = widgets.Text(value='agents_0309_9_55pm.xlsx')
display(agents_name)

Text(value='agents_0309_9_55pm.xlsx')

In [11]:
path_to_agents = "Source/"+ agents_name.value
agent_ref = pd.read_excel(path_to_agents)

In [12]:
agent_ref.tail()

,Unnamed: 0,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
89,89,7,211_11,21620,4253837,1276151.1,6486.1282,36,0,0,32,1,89,Young,Native,Rent,2,High Income
90,90,8,211_11,23541,2575160,772548.0,7062.4282,67,0,1,42,1,90,Old,Native,Rent,2,High Income
91,91,9,211_11,23490,4736181,1420854.3,7047.1282,39,0,0,33,1,91,Young,Native,Rent,2,High Income
92,92,10,211_11,22139,3070365,921109.5,6641.8282,76,0,1,16,1,92,Old,Native,Rent,2,High Income
93,93,11,211_11,7527,1961759,588527.7,2258.3282,102,1,1,42,1,93,Old,Native,Owner,0,Low Income


<h2>Files have been loaded</h2> 
<h1>step 2:Displyaing the the original buildings Using Folium and displaying statistics to them</h1>

In [13]:
# get the relevent columns for the buildings
script_file.columns

Index(['index', 'bld_address', 'OriginalUnits', 'OriginalFloors',
       'OriginalHouseSize', 'purchase_p', 'rent_price', 'Maintenace',
       'ArnonaPerMeter', 'Arnona', 'Old_Purchase', 'OldRent',
       'Tax and Maintenace', 'East', 'North', 'bld_operation', 'pointPolygon',
       'ParcelChange', 'FuturePlanID', 'ExcelBefore', 'ExcelChange',
       'ExcelPolygon', 'GIS_files', 'Address Title', 'TypeTitle'],
      dtype='object')

In [14]:
bldg_list = script_file[script_file['pointPolygon']=='Point'][['bld_address', 'OriginalUnits', 'OriginalFloors',
       'OriginalHouseSize', 'purchase_p', 'rent_price', 'Arnona', 'Old_Purchase', 'OldRent',
       'Tax and Maintenace']]

In [15]:
list_of_excels = script_file[script_file['pointPolygon']=='Polygon']['ExcelPolygon']

In [16]:
for i in list_of_excels.index:
    next_complex = list_of_excels[i]
    path_xlsx = data['bldg_path']+next_complex
    script_of_complex = pd.read_excel(path_xlsx)
    destroyed_buildings = script_of_complex[script_of_complex['status']=='destroyed']
    append_to = destroyed_buildings[['bld_address', 'OriginalUnits', 'OriginalFloors',
       'OriginalHouseSize', 'purchase_p', 'rent_price', 'Arnona', 'Old_Purchase', 'OldRent',
       'Tax and Maintenace']]
    bldg_list = bldg_list.append(append_to)
bldg_list.head(3)

,bld_address,OriginalUnits,OriginalFloors,OriginalHouseSize,purchase_p,rent_price,Arnona,Old_Purchase,OldRent,Tax and Maintenace
0,215_22,18,3.0,88,22930,62,478,2017840,6034,578
1,216_10,12,3.0,123,24376,68,668,2998248,9132,768
2,216_4,12,3.0,94,23351,67,510,2194994,6908,610


## Joining the GIS and this list

In [17]:
old_bldgs.head(2)

,Area,floors,height,bld_num,newUnits,polyID,bld_row_num,rowNum,bld_id,OriginalUnits,AdditionalUnits,TotalUnits,geometry
0,380.365771,2,6.0,None,8.0,3,53,53,212_59,NaN,NaN,NaN,"POLYGON ((175626.182 659083.328, 175633.042 65..."
1,435.489811,4,12.0,None,16.0,6,50,50,212_61,NaN,NaN,NaN,"POLYGON ((175615.252 659026.518, 175616.122 65..."


In [18]:
select_bldg = old_bldgs['bld_id'].isin(bldg_list.bld_address)

In [19]:
select_bldg_ds = old_bldgs[select_bldg][['bld_id','geometry']]

In [20]:
select_bldg_ds.reset_index(inplace=True)
select_bldg_ds

,index,bld_id,geometry
0,4,216_4,"POLYGON ((175700.912 659079.848, 175698.202 65..."
1,5,216_10,"POLYGON ((175650.672 659023.898, 175652.932 65..."
2,27,215_22,"POLYGON ((175781.082 658985.788, 175780.982 65..."
3,49,211_9,"POLYGON ((175829.692 659597.548, 175819.222 65..."
4,55,209_6,"POLYGON ((175949.832 659683.268, 175953.802 65..."
5,60,210_18,"POLYGON ((175843.402 659616.768, 175846.572 65..."
6,61,210_16,"POLYGON ((175869.862 659643.178, 175869.942 65..."
7,62,211_11,"POLYGON ((175837.492 659595.498, 175837.302 65..."
8,65,211_7,"POLYGON ((175799.652 659608.288, 175807.622 65..."


In [22]:
    def Convert_2039_2_4326(feature):
        #Convert From ESPG 2039 (Israel TM Grid) to ESPG 4326 (WGS 84)
        try:
            #step 1 
            interFeature = feature.to_crs("+proj=tmerc +lat_0=31.7343936111111 +lon_0=35.2045169444445 +k=1.0000067 +x_0=219529.584 +y_0=626907.39 +ellps=GRS80 +towgs84=-23.772,-17.490,-17.859,-0.31320,-1.85274,1.67299,5.4262 +units=m +no_defs")
            crs_4326 =  {'init': 'epsg:4326'}
            #step 2
            Feature_4326 = interFeature.to_crs(crs_4326)
            return (Feature_4326)

        except Exception as e:
            print(e)
        else:
            print("Conversion to WGS 84 Successful")

In [23]:
select_bldg_ds_join = pd.merge(select_bldg_ds,bldg_list,how='left',left_on='bld_id',right_on='bld_address')
select_bldg_wgs_84 = Convert_2039_2_4326(select_bldg_ds_join)

### get center point

In [24]:
lon = select_bldg_wgs_84.geometry.centroid.x.mean()
lat = select_bldg_wgs_84.geometry.centroid.y.mean()

latlng=[lat, lon]
zoom = 16
fmap = folium.Map(latlng,zoom_start=zoom,tiles='cartodbpositron')
fmap2 = folium.Map(latlng,zoom_start=zoom,tiles='cartodbpositron')

### add background buildings

In [25]:
old_all_bldgs_wgs84 = Convert_2039_2_4326(old_all_bldgs)

In [26]:
gjson = old_all_bldgs_wgs84.to_json()
style_1 ={'fillOpacity': 0.25,'weight': 0,'fillColor': '##cc0000'}
folium.GeoJson(gjson,
               style_function = lambda x: style_1).add_to(fmap)
folium.GeoJson(gjson,
               style_function = lambda x: style_1).add_to(fmap2)
display(fmap)

### add building we will work on simulation

In [27]:
gjson = select_bldg_wgs_84.to_json()
style_2 ={'fillOpacity': 0.75,'weight': 1,'fillColor': '#ff0000'}
folium.GeoJson(gjson,
               style_function = lambda x: style_2,
               tooltip=folium.GeoJsonTooltip(fields=['bld_address',
                                                     'OriginalUnits',
                                                     'OriginalFloors',
                                                     'OriginalHouseSize',
                                                     'purchase_p',
                                                     'rent_price',
                                                     'Arnona',
                                                     'Old_Purchase', 
                                                     'OldRent',
                                                     'Tax and Maintenace'
                                                     ],
                                             aliases=['Building Address Code',
                                                     'Number of Units',
                                                     'OriginalFloors',
                                                     'Average house size',
                                                     'Price Per Meter',
                                                     'Rent Per Meter',
                                                     'Arnona',
                                                     'Average Sell Price', 
                                                     'Average Rent Price',
                                                     'Tax and Maintenace'                                                   
                                                      ])
              ).add_to(fmap)
display(fmap)

In [28]:
agent_ref.head(10)


,Unnamed: 0,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
0,0,0,215_22,13842,697698,209309.4,4152.8504,98,1,1,37,1,0,Old,Native,Owner,1,Medium Income
1,1,1,215_22,7319,431148,129344.4,2195.9504,53,1,0,45,1,1,Young,Native,Owner,0,Low Income
2,2,2,215_22,14488,2666059,799817.7,4346.6504,73,1,1,30,1,2,Old,Native,Owner,1,Medium Income
3,3,3,215_22,7655,4475813,1342743.9,2296.7504,108,1,1,39,1,3,Old,Native,Owner,0,Low Income
4,4,4,215_22,5067,4651847,1395554.1,1520.3504,40,1,0,39,1,4,Young,Native,Owner,0,Low Income
5,5,5,215_22,22090,126175,37852.5,6627.1504,97,0,1,35,1,5,Old,Native,Rent,2,High Income
6,6,6,215_22,13724,8303,2490.9,4117.4504,96,1,1,33,1,6,Old,Native,Owner,1,Medium Income
7,7,7,215_22,22580,1104835,331450.5,6774.1504,91,0,1,13,1,7,Old,Native,Rent,2,High Income
8,8,8,215_22,12537,2544417,763325.1,3761.3504,54,1,0,32,1,8,Young,Native,Owner,1,Medium Income
9,9,9,215_22,23076,3169484,950845.2,6922.9504,91,0,1,41,1,9,Old,Native,Rent,2,High Income


In [29]:
agent_mod = agent_ref.copy()

In [30]:
#Unpivot
#Age
agent_mod.loc[agent_mod['AgeGroupDesc']=='Young','Young']=True
agent_mod.loc[agent_mod['AgeGroupDesc']=='Old','Old']=True

#ownership
agent_mod.loc[agent_mod['OwnershipDesc']=='Owner','Owner']=True
agent_mod.loc[agent_mod['OwnershipDesc']=='Rent','Rent']=True


#Income
agent_mod.loc[agent_mod['IncomeDesc']=='Low Income','Low Income']=True
agent_mod.loc[agent_mod['IncomeDesc']=='Medium Income','Medium Income']=True
agent_mod.loc[agent_mod['IncomeDesc']=='High Income','High Income']=True


In [31]:
agent_gb = agent_mod.groupby('AgentBldAdd')

In [32]:
agents_stat = agent_gb.agg({'AgentIncome':'mean','AgentWealth':'mean','Age':'mean','Native_Seniority':'mean','Young':'count','Old':'count','Owner':'count','Rent':'count','Low Income':'count','Medium Income':'count','High Income':'count'})
#,'Young':'count'

In [33]:
agents_stat.reset_index(inplace=True)

In [34]:
agents_stat

,AgentBldAdd,AgentIncome,AgentWealth,Age,Native_Seniority,Young,Old,Owner,Rent,Low Income,Medium Income,High Income
0,209_6,17378.666667,3.090857e+06,70.000000,28.500000,3,3,3,3,0,3,3
1,210_16,12999.894737,2.221719e+06,63.368421,27.315789,9,10,9,10,4,15,0
2,210_18,15978.083333,2.458109e+06,60.166667,28.666667,7,5,6,6,1,8,3
3,211_11,19377.833333,2.926607e+06,59.583333,26.666667,7,5,3,9,1,2,9
4,211_7,15336.000000,2.913312e+06,76.500000,23.500000,0,2,1,1,0,1,1
5,211_9,15890.000000,4.684045e+06,41.000000,30.000000,1,0,1,0,0,1,0
6,215_22,14915.444444,2.351387e+06,77.500000,28.888889,6,12,13,5,4,9,5
7,216_10,18282.250000,2.262696e+06,79.583333,26.083333,3,9,8,4,2,6,4
8,216_4,14192.666667,2.254924e+06,72.166667,28.666667,5,7,8,4,6,2,4


In [35]:
agents_stat['AgentIncome'] = agents_stat['AgentIncome'].astype(int)
agents_stat['AgentWealth'] = agents_stat['AgentWealth'].astype(int)
agents_stat['Native_Seniority'] = agents_stat['Native_Seniority'].astype(int)
agents_stat['Age'] = agents_stat['Age'].astype(int)

In [36]:
select_bldg_slim = select_bldg_wgs_84[['geometry','bld_id']]

In [37]:
bldg_with_agentStat = pd.merge(select_bldg_slim,agents_stat,how='left',left_on='bld_id',right_on='AgentBldAdd')


In [38]:
bldg_with_agentStat.columns

Index(['geometry', 'bld_id', 'AgentBldAdd', 'AgentIncome', 'AgentWealth',
       'Age', 'Native_Seniority', 'Young', 'Old', 'Owner', 'Rent',
       'Low Income', 'Medium Income', 'High Income'],
      dtype='object')

In [39]:
gjson = bldg_with_agentStat.to_json()
style_3 ={'fillOpacity': 0.75,'weight': 1,'fillColor': '#ff0000'}
folium.GeoJson(gjson,
               style_function = lambda x: style_3,
               tooltip=folium.GeoJsonTooltip(fields=['bld_id',
                                                     'AgentIncome',
                                                     'AgentWealth',
                                                     'Age',
                                                     'Native_Seniority',
                                                     'Young',
                                                     'Old',
                                                     'Owner',
                                                     'Rent',
                                                     'Low Income',
                                                     'Medium Income',
                                                     'High Income'
                                                     ],
                                             aliases=['Building Address Code',
                                                      'Average Agent income',
                                                     'Average Agent Wealth',
                                                     'Average Age',
                                                     'Average Seniority in Neighborhood',
                                                     'Number of Young',
                                                     'Number of Old',
                                                     'Number of Owners',
                                                     'Number of Rent',
                                                     'Number of Low Income',
                                                     'Number of Medium Income',
                                                     'Number of High Income'                                                  
                                                      ])
              ).add_to(fmap2)
display(fmap2)

In [268]:
fmap2.save('iteration0.html')

In [270]:
fmap.save('iteration0_bld_info.html')

<h1>step 3: setting up first iteration of simulation</h1>

In [40]:
#copying the original Agents
agent_0 = agent_ref.copy()

<h2>Adding Threshold ability to stay for each agent and initalStatus<h2/>

In [41]:
#cleanup
agent_0.columns

Index(['Unnamed: 0', 'AgentAppUnit', 'AgentBldAdd', 'AgentIncome',
       'AgentWealth', 'AgentPurchaseThreshold', 'AgentRentThreshold', 'Age',
       'OwnerShip', 'AgeGroup', 'Native_Seniority', 'Native_Group', 'AgentID',
       'AgeGroupDesc', 'NativeDesc', 'OwnershipDesc', 'IncomeNum',
       'IncomeDesc'],
      dtype='object')

In [42]:
agent_0 = agent_0[['AgentAppUnit', 'AgentBldAdd', 'AgentIncome',
       'AgentWealth', 'AgentPurchaseThreshold', 'AgentRentThreshold', 'Age',
       'OwnerShip', 'AgeGroup', 'Native_Seniority', 'Native_Group', 'AgentID',
       'AgeGroupDesc', 'NativeDesc', 'OwnershipDesc', 'IncomeNum',
       'IncomeDesc']]

In [43]:
agent_0.head()

,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
0,0,215_22,13842,697698,209309.4,4152.8504,98,1,1,37,1,0,Old,Native,Owner,1,Medium Income
1,1,215_22,7319,431148,129344.4,2195.9504,53,1,0,45,1,1,Young,Native,Owner,0,Low Income
2,2,215_22,14488,2666059,799817.7,4346.6504,73,1,1,30,1,2,Old,Native,Owner,1,Medium Income
3,3,215_22,7655,4475813,1342743.9,2296.7504,108,1,1,39,1,3,Old,Native,Owner,0,Low Income
4,4,215_22,5067,4651847,1395554.1,1520.3504,40,1,0,39,1,4,Young,Native,Owner,0,Low Income


In [46]:
#agent_0['TaxAndMainTreshold'] = agent_0['AgentIncome']*0.3
#agent_0['RentTreshold'] = agent_0['AgentIncome']*0.3
#agent_0['BuyTreshold'] = agent_0['AgentWealth']*0.3
#agent_0['Agent_status'] = 'staying' #staying, leaving, buying 
agent_0 = updateAgentsThresholdData(agent_0,'staying')

In [47]:
def updateAgentsThresholdData(ds,status):
    #This function recieves a Agent Dataset
    #The function returns the agents ability to live in a certain building as column data
    ds['TaxAndMainTreshold'] = ds['AgentIncome']*0.3
    ds['RentTreshold'] = ds['AgentIncome']*0.3
    ds['BuyTreshold'] = ds['AgentWealth']*0.3
    ds['Agent_status'] = status #staying, leaving,newComers buying always starts with staying
    return ds

In [53]:
agent_0.head()

,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,...,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc,TaxAndMainTreshold,RentTreshold,BuyTreshold,Agent_status
0,0,215_22,13842,697698,209309.4,4152.8504,98,1,1,37,...,0,Old,Native,Owner,1,Medium Income,4152.6,4152.6,209309.4,staying
1,1,215_22,7319,431148,129344.4,2195.9504,53,1,0,45,...,1,Young,Native,Owner,0,Low Income,2195.7,2195.7,129344.4,staying
2,2,215_22,14488,2666059,799817.7,4346.6504,73,1,1,30,...,2,Old,Native,Owner,1,Medium Income,4346.4,4346.4,799817.7,staying
3,3,215_22,7655,4475813,1342743.9,2296.7504,108,1,1,39,...,3,Old,Native,Owner,0,Low Income,2296.5,2296.5,1342743.9,staying
4,4,215_22,5067,4651847,1395554.1,1520.3504,40,1,0,39,...,4,Young,Native,Owner,0,Low Income,1520.1,1520.1,1395554.1,staying


<h2>Here I am initating datsets for referncing the differnet excel sheets in each iteration</h2>

In [50]:
# creating a dateset for the excel files that will be saved
itr_agents_excel_ds = pd.DataFrame()
itr_bldg_excel_ds = pd.DataFrame()
itr_accumulated_bldg_excel_ds = pd.DataFrame()
itr_geokpackage_ds = pd.DataFrame()

<h2>lets give a name to this simulation and control the iteration</h2>

In [52]:
from datetime import datetime
dt = datetime.now()
d1 = dt.strftime("%m%d_%H%M")
sim_name="cim_"+d1
simulation_name = widgets.Text(value=sim_name)
display(simulation_name)


Text(value='cim_0310_1945')

<h2>controlling iteration</h2>

In [55]:

itr_slider = widgets.IntSlider(min=0, max =6)
display(itr_slider)

IntSlider(value=0, max=6)

<h1 >TODO 1. !!! Need to create a flat file with a description of all the projects for each building</h1>

<h1>step 4: starting iteration</h1>

In [1106]:
i = itr_slider.value
print("Iteration Number: " + str(i))

Iteration Number: 4


<h1>step 4.1: grabing the next row in script file, and getting a refenrence of the excel file</h1>

In [1127]:
# reference of the next row
next_row = script_file.loc[i]
after_change_file = next_row['ExcelChange']
bld_add_or_complex = next_row['bld_address']
typeProject = next_row['TypeTitle']
typeOperation = next_row['bld_operation']
excel_path = data['bldg_path']+after_change_file

updated_bldg = pd.read_excel(excel_path)

ep_originalbldg = data['bldg_path']+next_row['ExcelBefore']
if (typeOperation==3):
    original_bldg = pd.read_excel(ep_originalbldg)
#print the name of the building refernece file
print('After Excel File: ' + after_change_file)
print('Address or complex: ' + bld_add_or_complex)
print('Type of project ' + typeProject)




After Excel File: after_complex_4.xlsx
Address or complex: complex_4
Type of project תב״ע פינוי בינוי


In [1112]:
next_row

index                                                   4
bld_address                                     complex_4
OriginalUnits                                          46
OriginalFloors                                        2.5
OriginalHouseSize                                      72
purchase_p                                          21336
rent_price                                             62
Maintenace                                            100
ArnonaPerMeter                                     5.4358
Arnona                                                391
Old_Purchase                                      1536192
OldRent                                              4955
Tax and Maintenace                                    491
East                                               175839
North                                              659613
bld_operation                                           3
pointPolygon                                      Polygon
ParcelChange  

<h2>Units ifnromation of the updated building:</h2>

In [1129]:
bldg_index = updated_bldg.loc[0].bldAdd #getting a refernec of the first building
if (typeOperation==3):
    list_of_bldgs = original_bldg.bldAdd.unique().tolist()
# in case of Taba this logic changes!
updated_bldg.tail()

,Unnamed: 0,index,appUnits,bldAdd,AppSize,sellPerMeter,rentPerMeter,ArnonaTarif,ArnonaTax,sellPrice,rentPerMonth,newMaintenace&Tax,newMaintenace
0,0,0,0,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
1,1,1,1,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
2,2,2,2,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
3,3,3,3,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
4,4,4,4,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
5,5,5,5,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
6,6,6,6,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
7,7,7,7,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
8,8,8,8,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
9,9,9,9,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building


In [1130]:
list_of_bldgs

['210_16', '210_18', '211_7', '211_9', '211_11']

<h1>step 2.2: duplicating the agent file</h1>

In [1124]:

if (i==0):
    agent_i = agent_0.copy()


<h1>step 2.3: merging the building details with the agent details</h1>

In [1125]:
typeOperation

3

In [1126]:
if (typeOperation)<3:
    agent_i_merge = pd.merge(agent_i,updated_bldg, how='left',left_on=['AgentBldAdd','AgentAppUnit'],right_on=['bldAdd','appUnits'])

<h1>step 4.2: Have a refernece of the active rows</h1>

In [1145]:
if (typeOperation)<3:
    active_rows = agent_i_merge['bldAdd']==bldg_index #in case of Taba Logic Changes
else:
    agent_i_merge = agent_i.copy() 
    active_rows = agent_i_merge.AgentBldAdd.isin(list_of_bldgs)

active_rows_renters =  (active_rows & (agent_i_merge['OwnerShip']==0)) #renters they all go!
active_rows_owners =  (active_rows & (agent_i_merge['OwnerShip']==1)) #owners go if their income is insufficent
agent_i_merge.loc[active_rows_renters,'Agent_status']='laeving'



in


<h1>step 4.3 set the simplest descion tree possible</h1>

## 1. if the agent is renting he will leave in all cases
## 2. if the agent is not renting, he will leave if he cannot pay monthly bills and maintances

In [1244]:
#active_rows_owners
if (typeOperation<3):
    agent_tax_ability = agent_i_merge[active_rows]['TaxAndMainTreshold']
    bld_expenses_price =  agent_i_merge[active_rows]['newMaintenace&Tax']

    agent_i_merge.loc[active_rows,'Can stay if own'] = (agent_tax_ability > bld_expenses_price)

    active_rows_owners_cannotStay =  (active_rows_owners & (agent_i_merge['Can stay if own']==False)) #both Owners who can't pay will leave
    agent_i_merge.loc[active_rows_owners_cannotStay,'Agent_status']='laeving'
else:
    # 1. get only active owners
    # 2. reset their index
    # 3. merge with new buidling
    # 4. check ability to pay rent
    # 5. update their active living
    # 5. overwrite agent_i_file using the agentID (delete them and append them)
    
    # 1. get only active owners
    
    activeOwnerAgents = agent_i_merge[active_rows_owners]
    
    # 2. reset their index
    activeOwnerAgents.reset_index(inplace=True) # so we can matched the new buildings to those agents
    activeOwnerAgents = activeOwnerAgents[['Age', 'AgeGroup', 'AgeGroupDesc', 'AgentAppUnit',
       'AgentBldAdd', 'AgentID', 'AgentIncome', 'AgentPurchaseThreshold',
       'AgentRentThreshold', 'AgentWealth', 'Agent_status', 'BuyTreshold',
       'IncomeDesc', 'IncomeNum', 'NativeDesc', 'Native_Group',
       'Native_Seniority', 'OwnerShip', 'OwnershipDesc', 'RentTreshold',
       'TaxAndMainTreshold', 'ones']]
    # 3. merge with new buidling
    activeOwnerAgents_merge = pd.merge(activeOwnerAgents,updated_bldg, how='left', left_index=True, right_index=True)
    # 4. check ability to pay rent
    agent_tax_ability = activeOwnerAgents_merge['TaxAndMainTreshold']
    bld_expenses_price =  activeOwnerAgents_merge['newMaintenace&Tax']
    activeOwnerAgents_merge['Can stay if own'] = (agent_tax_ability > bld_expenses_price)
    # 5. update their active living
    survived_agentsBool =  activeOwnerAgents_merge['Can stay if own']==True
    leavingAgentsBool =  activeOwnerAgents_merge['Can stay if own']==False
    activeOwnerAgents_merge.loc[leavingAgentsBool,['Agent_status']]='leaving'
    
    update_leave_agents_list = activeOwnerAgents_merge[leavingAgentsBool].AgentID.to_list()
    agent_i_merge.loc[agent_i_merge.AgentID.isin(update_leave_agents_list),['Agent_status']]='leaving' #update leaving agents
    
    survived_agents_update = activeOwnerAgents_merge[survived_agentsBool][['AgentID','AgentBldAdd','AgentAppUnit']]
    updated_bldg_slim = updated_bldg[['bldAdd','appUnits']]
    # Change the address of the agents
    newAddress_for_agents = pd.merge(survived_agents_update,updated_bldg_slim, how='left', left_index=True, right_index=True)
    newAddress_for_agents = newAddress_for_agents[['AgentID','bldAdd','appUnits']]
    #update agents Pool
    change_agent_bool = agent_i_merge.AgentID.isin(newAddress_for_agents['AgentID'].tolist())
    agent_i_merge.loc[change_agent_bool,'AgentBldAdd'] = newAddress_for_agents.bldAdd.tolist()
    agent_i_merge.loc[change_agent_bool,'AgentAppUnit'] = newAddress_for_agents.appUnits.tolist()
    #select all the units left
    next_row = max(newAddress_for_agents.index)  + 1
    avilable_unitsin_bldg = updated_bldg.iloc[next_row:] #rows for constructing new agents

In [1245]:
#adding a building and door code
if (typeOperation<3):
    Current_residents_who_stay_rows = (active_rows & (agent_i_merge['Agent_status']=="staying"))
    crwsr = Current_residents_who_stay_rows #short writing
    staying_agents = agent_i_merge[crwsr][['AgentAppUnit','AgentBldAdd']]
    staying_agents['add_door'] = staying_agents['AgentBldAdd']+"_"+staying_agents['AgentAppUnit'].astype(str)
    #adding a building and door code

    updated_bldg['add_door_2'] = updated_bldg['bldAdd']+"_"+updated_bldg['appUnits'].astype(str)
    avilable_units = ~updated_bldg.add_door_2.isin(staying_agents['add_door']) #avilable units

<h1>step 4.5: getting refernece of avilable appartments and generating new agents (or grabing them from other sources)</h1>

In [1247]:
if (typeOperation<3):
    avilable_unitsin_bldg = updated_bldg[avilable_units]
avilable_unitsin_bldg.reset_index(inplace=True)
avilable_unitsin_bldg.head()

,level_0,Unnamed: 0,index,appUnits,bldAdd,AppSize,sellPerMeter,rentPerMeter,ArnonaTarif,ArnonaTax,sellPrice,rentPerMonth,newMaintenace&Tax,newMaintenace
0,20,20,20,20,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
1,21,21,21,21,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
2,22,22,22,22,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
3,23,23,23,23,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building
4,24,24,24,24,210_8,120,26670,80.6,5.4358,652.296,3200400,10774.296,1102.296,New Building


In [1248]:
newAgents = generateAgensts(avilable_unitsin_bldg)
agentsCol = newAgents.columns.tolist()
agentsCol.append('AgentID')
#updating the agents file
agent_i = agent_i_merge[agentsCol]
agent_i = agent_i.append(newAgents)
agent_i.reset_index(inplace=True)
agent_i['AgentID'] = agent_i.index

/Users/shai/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


<h1>step 5. Saving the excel to a file and appending it into a a new dataset</h2>

In [1251]:
agent_i['ones'] = 1
pivot_status = agent_i.pivot(index='AgentID',columns='Agent_status',values='ones')

del pivot_status.index.name

agent_i_export = agent_i.copy()
agent_i_export[['laeving','new comers','staying']] = pivot_status
#remeber to remove this pivot before next iteration
agent_i_export['iteration']=str(i)

## save the agents file to a new folder the project folder

In [1252]:
simulation_path = "simulationID/"+simulation_name.value+"/"
if (i==0):
  os.mkdir(simulation_path)

save_iteration = 'agents_itr_'+str(i)+'.xlsx'
excel_path = simulation_path+save_iteration
agent_i_export.to_excel(excel_path)

if (i==0):
    #save a copy of Agents 0 for statistics
    agent_0.to_excel(simulation_path + 'agent_0.xlsx')


<h1>Step 6. calcualting otther sattistics and saving them to excel files</h1>

In [1253]:
itr_agents_excel_ds.loc[i,'Excel_File']=excel_path
itr_agents_excel_ds.loc[i,'Bldg Project or Address']=bld_add_or_complex

In [1254]:
gb = agent_i_export.groupby('AgentBldAdd')
agg1_p = gb.agg({'laeving':'count','new comers':'count','staying':'count'})
agg1_p = agg1_p.reset_index()
agg1_p

,AgentBldAdd,laeving,new comers,staying
0,209_6,3,22,3
1,210_16,10,0,0
2,210_18,6,0,0
3,210_8,0,88,20
4,211_11,9,0,0
5,211_7,1,0,0
6,211_9,0,136,0
7,215_22,5,20,13
8,216_10,4,12,8
9,216_4,4,14,8


In [1255]:
#agg1 = agg1[agg1['AgentBldAdd'] == bld_add_or_complex] # need to fix to take care of complex projects

In [1256]:
agg1_p['New Total Residence'] = agg1_p['new comers'] + agg1_p['staying']
agg1_p['Old Total Residence'] = agg1_p['laeving'] + agg1_p['staying']
agg1_p['Renewd'] = agg1_p['New Total Residence']>agg1_p['Old Total Residence'] 
agg1_p['Type of Project'] = typeProject #incorrect!!!! need to convert to list
agg1_p['Percent Left'] = agg1_p['laeving'] /agg1_p['Old Total Residence']
agg1_p['Percent New Comers'] = agg1_p['new comers'] /agg1_p['New Total Residence']
agg1 = agg1_p[agg1_p['Renewd']==True]
agg1.reset_index(inplace=True)


In [1257]:
agg1

,index,AgentBldAdd,laeving,new comers,staying,New Total Residence,Old Total Residence,Renewd,Type of Project,Percent Left,Percent New Comers
0,0,209_6,3,22,3,25,6,True,תב״ע פינוי בינוי,0.500000,0.880000
1,3,210_8,0,88,20,108,20,True,תב״ע פינוי בינוי,0.000000,0.814815
2,6,211_9,0,136,0,136,0,True,תב״ע פינוי בינוי,NaN,1.000000
3,7,215_22,5,20,13,33,18,True,תב״ע פינוי בינוי,0.277778,0.606061
4,8,216_10,4,12,8,20,12,True,תב״ע פינוי בינוי,0.333333,0.600000
5,9,216_4,4,14,8,22,12,True,תב״ע פינוי בינוי,0.333333,0.636364


In [1258]:
temp = (agg1[['laeving','new comers','staying','New Total Residence','Old Total Residence']].sum())

In [1259]:
temp

laeving                 16
new comers             292
staying                 52
New Total Residence    344
Old Total Residence     68
dtype: int64

In [1260]:
 newsum = pd.DataFrame(temp).transpose()

In [1261]:
newsum['itr'] = i
newsum['Percent Left'] = newsum['laeving'] /newsum['Old Total Residence']
newsum['Percent New Comers'] = newsum['new comers'] /newsum['New Total Residence']

In [1262]:
if (i==0):
    agg2 = newsum
else:
    agg2.loc[i] = newsum.loc[0]    #agg2.loc[i,['itr']]=i
    #agg2.loc[i,'itr'] =i
    #agg2.loc[i,'Percent Left']] = newsum['laeving'] /newsum['Old Total Residence']
    #newsum['Percent New Comers'] = newsum['new comers'] /newsum['New Total Residence']



In [1263]:

save_iteration = 'bldg_itr_'+str(i)+'.xlsx'
excel_path = simulation_path+save_iteration
agg1.to_excel(excel_path)
itr_bldg_excel_ds.loc[i,'Excel_File']=excel_path
itr_bldg_excel_ds.loc[i,'BldgOrProjectName']=bld_add_or_complex

save_iteration = 'accumulated_'+save_iteration
accumulated_excel_path=simulation_path+save_iteration
agg2.to_excel(accumulated_excel_path)


if (i>0):
  print(i)
  itr_bldg_excel_ds.loc[i,'Excel_File']=accumulated_excel_path
  itr_bldg_excel_ds.loc[i,'BldgOrProjectName']=bld_add_or_complex
else:
  itr_accumulated_bldg_excel_ds = itr_bldg_excel_ds

4


## Need to Add Geographic Representation and saving
## Need to Add Plotly

In [1264]:
itr_agent_path = simulation_path+'agents_iteration_excels.xlsx'
itr_bldg_path = simulation_path+'bldg_iteration_excels.xlsx'
#itr_geopacakge_path = simulation_path+'iteration_geopackage.xlsx'
accum_path = simulation_path+'itr_accumnulated_bldg.xlsx'
itr_bldg_excel_ds.to_excel(itr_bldg_path)
itr_agents_excel_ds.to_excel(itr_agent_path)
#itr_geokpackage_ds.to_excel(itr_geopacakge_path)
itr_accumulated_bldg_excel_ds.to_excel(accum_path)

<h1 >TODO 1. !!! Need to create a flat file with a description of all the projects for each building</h1>

In [679]:
def generateAgensts(thisDS):
    #this function recievs data from new building dataset and returns new renters or owners
    nUnits = len(thisDS['appUnits'])

    rand_income_rent = np.random.randint(0, high=5000, size=nUnits, dtype='l') #monthly income
    rand_inc_sr_rent =  pd.Series(rand_income_rent) #turning this into series

    rand_income_own = np.random.randint(3000, high=15000, size=nUnits, dtype='l') #monthly income
    rand_inc_sr_own =  pd.Series(rand_income_own) #into series
    
    rand_wealth = np.random.randint(0, high=5000000, size=nUnits, dtype='l') #wealth
    age = np.random.randint(25, high=110, size=nUnits, dtype='l') #wealth
    ownership = np.random.randint(0, high=2, size=nUnits, dtype='l') #0 rent 1 owns
    
    ds = pd.DataFrame()
    
    ds['OwnerShip'] = pd.Series(ownership)
    ds['AgentAppUnit'] = thisDS['appUnits'] #what unit does the agent live in
    ds['AgentBldAdd'] = thisDS['bldAdd'] # in what bld address does the Agent live in

 
    ds['AgentWealth'] = pd.Series(rand_wealth)
    
    ds['AgentPurchaseThreshold'] = (ds['AgentWealth'] * 0.3).astype(int) # 30% of wealth, Agent can buy a house without Mortgage
    ds['Age'] = pd.Series(age)
    
    ds['AgeGroup'] = ds['Age'].apply(lambda x: 0 if x < 65 else 1)
    ds['Native_Seniority'] = 0
    ds['Native_Group'] = ds['Native_Seniority'].apply(lambda x: 0 if x < 10 else 1) # 0 Native, 1 NewComer
    
    # for renters take the rent price
    ds.loc[ds['OwnerShip']==0,'AgentIncome'] = (thisDS[ds['OwnerShip']==0]['rentPerMonth'])/0.3 + rand_inc_sr_rent[ds['OwnerShip']==0]
    ds.loc[ds['OwnerShip']==1,'AgentIncome'] = (thisDS[ds['OwnerShip']==1]['newMaintenace&Tax'])/0.3 + rand_inc_sr_own[ds['OwnerShip']==1]
    ds['AgentIncome'] = ds['AgentIncome'].astype(int)
    ds['AgentRentThreshold'] = (ds['AgentIncome'] * 0.3).astype(int) # 30% of income
    
    #Age Formating
    ds['AgeGroupDesc'] = ds['Age'].apply(lambda x: 'Young' if x < 65 else 'Old')
    #Native Formating
    ds['NativeDesc'] = ds['Native_Group'].apply(lambda x: 'Native' if x == 1 else 'New Comers')
    #Ownership Formating
    ds['OwnershipDesc'] = ds['OwnerShip'].apply(lambda x: 'Owner' if x == 1 else 'Rent')
    
    LowInc = int(rules_1['Income']['Low']['0'])
    MediumInc = int(rules_1['Income']['Medium']['1'])
    HighInc = int(rules_1['Income']['High']['2'])
    
    ds['IncomeNum'] = ds['AgentIncome'].apply(lambda x: "0" if x < LowInc else "1" if x < MediumInc else "2")
    ds['IncomeDesc'] = ds['AgentIncome'].apply(lambda x: 'Low Income' if x < LowInc else 'Medium Income' if x < MediumInc else "High Income")
    ds = updateAgentsThresholdData(ds,'new comers')

    return ds




# Useful Pivot Operations (Reference)

In [412]:
agent_ref.head(4)

,Unnamed: 0,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
0,0,0,215_22,13842,697698,209309.4,4152.8504,98,1,1,37,1,0,Old,Native,Owner,1,Medium Income
1,1,1,215_22,7319,431148,129344.4,2195.9504,53,1,0,45,1,1,Young,Native,Owner,0,Low Income
2,2,2,215_22,14488,2666059,799817.7,4346.6504,73,1,1,30,1,2,Old,Native,Owner,1,Medium Income
3,3,3,215_22,7655,4475813,1342743.9,2296.7504,108,1,1,39,1,3,Old,Native,Owner,0,Low Income


In [446]:
agg_by_bldg = agent_ref.pivot_table(index=['AgentBldAdd'],columns='IncomeDesc',values='AgentIncome',aggfunc='count')
agg_by_bldg.head()

IncomeDesc,High Income,Low Income,Medium Income
AgentBldAdd,,,
209_6,3.0,NaN,3.0
210_16,NaN,4.0,15.0
210_18,3.0,1.0,8.0
211_11,9.0,1.0,2.0
211_7,1.0,NaN,1.0


In [447]:
agg_by_bldg.reset_index(inplace=True)

In [448]:
agg_by_bldg.columns

Index(['AgentBldAdd', 'High Income', 'Low Income', 'Medium Income'], dtype='object', name='IncomeDesc')

In [449]:
agg_by_bldg.columns.rename("row",inplace=True)

In [450]:
agg_by_bldg.head()

row,AgentBldAdd,High Income,Low Income,Medium Income
0,209_6,3.0,NaN,3.0
1,210_16,NaN,4.0,15.0
2,210_18,3.0,1.0,8.0
3,211_11,9.0,1.0,2.0
4,211_7,1.0,NaN,1.0


In [451]:
agg_by_bldg.set_index('AgentBldAdd', inplace=True)

In [452]:
agg_by_bldg.head()

row,High Income,Low Income,Medium Income
AgentBldAdd,,,
209_6,3.0,NaN,3.0
210_16,NaN,4.0,15.0
210_18,3.0,1.0,8.0
211_11,9.0,1.0,2.0
211_7,1.0,NaN,1.0


In [453]:
del agg_by_bldg.index.name

In [454]:
agg_by_bldg

row,High Income,Low Income,Medium Income
209_6,3.0,NaN,3.0
210_16,NaN,4.0,15.0
210_18,3.0,1.0,8.0
211_11,9.0,1.0,2.0
211_7,1.0,NaN,1.0
211_9,NaN,NaN,1.0
215_22,5.0,4.0,9.0
216_10,4.0,2.0,6.0
216_4,4.0,6.0,2.0


In [352]:
agent_ref.head(5)

,Unnamed: 0,AgentAppUnit,AgentBldAdd,AgentIncome,AgentWealth,AgentPurchaseThreshold,AgentRentThreshold,Age,OwnerShip,AgeGroup,Native_Seniority,Native_Group,AgentID,AgeGroupDesc,NativeDesc,OwnershipDesc,IncomeNum,IncomeDesc
0,0,0,215_22,13842,697698,209309.4,4152.8504,98,1,1,37,1,0,Old,Native,Owner,1,Medium Income
1,1,1,215_22,7319,431148,129344.4,2195.9504,53,1,0,45,1,1,Young,Native,Owner,0,Low Income
2,2,2,215_22,14488,2666059,799817.7,4346.6504,73,1,1,30,1,2,Old,Native,Owner,1,Medium Income
3,3,3,215_22,7655,4475813,1342743.9,2296.7504,108,1,1,39,1,3,Old,Native,Owner,0,Low Income
4,4,4,215_22,5067,4651847,1395554.1,1520.3504,40,1,0,39,1,4,Young,Native,Owner,0,Low Income


In [455]:
agg_by_bldg1 = agent_ref.pivot(index='AgentID',columns='IncomeDesc',values='AgentIncome')



In [458]:
agg_by_bldg1.head()

IncomeDesc,High Income,Low Income,Medium Income
AgentID,,,
0,NaN,NaN,13842.0
1,NaN,7319.0,NaN
2,NaN,NaN,14488.0
3,NaN,7655.0,NaN
4,NaN,5067.0,NaN


In [460]:
del agg_by_bldg1.index.name

AttributeError: name

In [462]:
agg_by_bldg1.head()

IncomeDesc,High Income,Low Income,Medium Income
0,NaN,NaN,13842.0
1,NaN,7319.0,NaN
2,NaN,NaN,14488.0
3,NaN,7655.0,NaN
4,NaN,5067.0,NaN
